<p style="background-color:#e24b32;font-family:newtimeroman;color:#e4e4e5;font-size:150%;text-align:center;border-radius:10px 10px;">Ship Classication Using Transfer Learning<p>

Problem Statement : https://unstop.com/hackathon/lt-edutech-hackathon-at-shaastra-iitm-shaastra-2023-indian-institute-of-technology-iit-madras-579093  
Dataset : https://cutt.ly/PS_3_dataset


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score,classification_report

import os
import glob
import cv2

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.xception import preprocess_input,Xception

import warnings
warnings.filterwarnings("ignore")

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# strategy = tf.distribute.MirroredStrategy()
# print('DEVICES AVAILABLE: {}'.format(strategy.num_replicas_in_sync))

FILE_DIR='/kaggle/input/game-of-deep-learning-ship-datasets/train/images'

In [2]:
submission=pd.read_csv('/kaggle/input/game-of-deep-learning-ship-datasets/sample_submission_ns2btKE.csv')
test=pd.read_csv('/kaggle/input/game-of-deep-learning-ship-datasets/test_ApKoW4T.csv')
train=pd.read_csv('/kaggle/input/game-of-deep-learning-ship-datasets/train/train.csv')

For Reference : `{0: 'Cargo', 1:'Military', 2:'Carrier', 3:'Cruise', 4:'Tankers'}`

In [3]:
d={}

for i in glob.glob(FILE_DIR+'/*.jpg'):
    img=cv2.imread(i)
    resulting_img = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    final_image=tf.keras.applications.xception.preprocess_input(resulting_img)
    d[os.path.basename(i)]=final_image

In [4]:
train_img_list=list(train.image)
test_img_list=list(test.image)

In [5]:
train_dic={k: d[k] for k in train_img_list} 
test_dic={k: d[k] for k in test_img_list} 

In [6]:
model_inputs=np.array(list(train_dic.values()))

In [7]:
train_labels=np.array(list(train.category))-1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(model_inputs,train_labels, test_size=0.1, random_state=42)

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
base_model=Xception(weights="imagenet",include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(5,activation='softmax')(avg)
model = tf.keras.Model(inputs=base_model.input,outputs=output)

In [ ]:
for layer in base_model.layers:
    layer.trainable=False
# with tpu_strategy.scope():
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])


# with strategy.scope():
history = model.fit(x=model_inputs,y=train_labels,epochs=5,validation_data=(X_val,y_val))

In [ ]:
for layer in base_model.layers:
    layer.trainable=True
# with tpu_strategy.scope():
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])

history = model.fit(x=model_inputs,y=train_labels,epochs=25,validation_data=(X_val,y_val))

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.show()

In [ ]:
y_pred_ = model.predict(X_test)
y_pred = np.argmax(y_pred_,axis=1)

In [ ]:
print(f"Classifiction Report :\n {classification_report(y_test,y_pred)}")

<p style="background-color:#e24b32;font-family:newtimeroman;color:#e4e4e5;font-size:150%;text-align:center;border-radius:10px 10px;">Judging Metric : Kappa Score<p> 

In [ ]:
kappa_scr=cohen_kappa_score(y_pred,y_test)
print(f"Kappa Score : {kappa_scr}")

### Predicting Values for given test dataset

In [ ]:
test_model_inputs=np.array(list(test_dic.values()))

In [ ]:
output_labels = model.predict(test_model_inputs)

In [ ]:
output_labels_ind = np.argmax(output_labels,axis=1)+1

In [ ]:
np.bincount(output_labels_ind)

In [ ]:
submission['category']=output_labels_ind

In [ ]:
submission.to_csv('submission_v1.csv',index=False)

<p style="background-color:#e24b32;font-family:newtimeroman;color:#e4e4e5;font-size:150%;text-align:center;border-radius:10px 10px;">Optimal Training dataset size<p> 

In [8]:
split_size_list=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
kappa_score=[]
model_logs = ['modeltrain8.h5','modeltrain7.h5','modeltrain6.h5','modeltrain5.h5','modeltrain4.h5','modeltrain3.h5','modeltrain2.h5','modeltrain1.h5']
flag=0

In [9]:
mirrored_strategy = tf.distribute.MirroredStrategy()

In [ ]:
for i in split_size_list:    
    X_train, X_test, y_train, y_test = train_test_split(model_inputs,train_labels, test_size=i, random_state=42)
    X_train,X_val,y_train,y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
    with mirrored_strategy.scope():
        base_model=Xception(weights="imagenet",include_top=False)
        avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
        output = tf.keras.layers.Dense(5,activation='softmax')(avg)
        model = tf.keras.Model(inputs=base_model.input,outputs=output)
        for layer in base_model.layers:
            layer.trainable=False
        # with tpu_strategy.scope():
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
        

        # with strategy.scope():
    history = model.fit(x=model_inputs,y=train_labels,epochs=3,validation_data=(X_val,y_val))
    model.save(model_logs[flag])
    with mirrored_strategy.scope():
        for layer in base_model.layers:
            layer.trainable=True
        # with tpu_strategy.scope():
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])

    history = model.fit(x=model_inputs,y=train_labels,epochs=3,validation_data=(X_val,y_val))
    
    model.save(model_logs[flag])
    flag+=1
    y_pred_ = model.predict(X_test)
    y_pred = np.argmax(y_pred_,axis=1)
    
    kpp=cohen_kappa_score(y_pred,y_test)
    print(f"Kappa Score {kpp}")
    kappa_score.append(kpp)

Epoch 1/3
196/196 [==============================] - 26s 56ms/step - loss: 1.1992 - accuracy: 0.4987 - val_loss: 0.9462 - val_accuracy: 0.6004
Epoch 2/3
196/196 [==============================] - 7s 34ms/step - loss: 0.8547 - accuracy: 0.6607 - val_loss: 0.7879 - val_accuracy: 0.7158
Epoch 3/3
196/196 [==============================] - 7s 35ms/step - loss: 0.7528 - accuracy: 0.7017 - val_loss: 0.7265 - val_accuracy: 0.7300
Epoch 1/3
196/196 [==============================] - 62s 179ms/step - loss: 0.6356 - accuracy: 0.7662 - val_loss: 0.1228 - val_accuracy: 0.9663
Epoch 2/3
196/196 [==============================] - 30s 153ms/step - loss: 0.1927 - accuracy: 0.9298 - val_loss: 0.0412 - val_accuracy: 0.9947
Epoch 3/3
196/196 [==============================] - 30s 152ms/step - loss: 0.0888 - accuracy: 0.9730 - val_loss: 0.0118 - val_accuracy: 0.9982
Kappa Score 1.0
Epoch 1/3
196/196 [==============================] - 18s 56ms/step - loss: 1.2018 - accuracy: 0.5045 - val_loss: 0.8857 - val

In [ ]:
plt.plot(np.array(split_size_list)*100,np.array(kappa_score))
plt.xlabel("Training Data %")
plt.ylabel("Metric - Kappa Score")
plt.show()

- Best percentage is when the model attain highest kappa score